# imports

In [27]:
import os
import cv2
import torch
import torchvision
! pip install torchsummary
! pip install pydicom
import torchsummary
from torch.utils.data import Dataset, DataLoader, Subset
import torch.nn as nn
from IPython.display import clear_output
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, roc_auc_score
from tqdm import tqdm
import numpy as np
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
from concurrent.futures import ProcessPoolExecutor 
import torch.nn.functional as F
import pydicom  
from tabulate import tabulate
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# config

In [28]:
class Config:
    # BASE_PATH = '/kaggle/input/rsna-2023-abdominal-trauma-detection'
    BASE_PATH = '.'
    TRAIN_IMG_PATH='rsna-2023-atd-reduced-256-5mm/reduced_256_tickness_5'
    SEED = 42
    IMAGE_SIZE = [256, 256]
    BATCH_SIZE = 25
    EPOCHS = 3
    DEPTH=300# median of number of instanceses is 270 ,mean=424
    NUM_FOLDS=5
    TARGET_COLS = [
       'bowel_healthy', 'bowel_injury', 'extravasation_healthy',
       'extravasation_injury', 'kidney_healthy', 'kidney_low', 'kidney_high',
       'liver_healthy', 'liver_low', 'liver_high', 'spleen_healthy',
       'spleen_low', 'spleen_high',
    ]

torch.manual_seed(Config.SEED)

# addaptation for depth

In [29]:
def standardize_depth(tensor, target_depth=300):
    # This function assumes tensor shape [depth, channel, height, width]
    depth = tensor.shape[0]

    if depth == target_depth:
        return tensor
    else:
        if tensor.dim() == 3:
            tensor = tensor.unsqueeze(0)  # Add a channel dimension
        return torch.nn.functional.interpolate(
            tensor.unsqueeze(0), 
            size=(target_depth, tensor.size(2), tensor.size(3)), 
            mode='trilinear', 
            align_corners=False
        ).squeeze(0).squeeze(0)

# dicom to png 

In [30]:
def dicom_to_img(dicom_image):
    dicom_image = pydicom.dcmread(dicom_image)
    pixel_array = dicom_image.pixel_array
    
    if dicom_image.PixelRepresentation == 1:
        bit_shift = dicom_image.BitsAllocated - dicom_image.BitsStored
        new_array = (pixel_array << bit_shift).astype(pixel_array.dtype) >>  bit_shift
        pixel_array = pydicom.pixel_data_handlers.util.apply_modality_lut(new_array, dicom_image)

    if dicom_image.PhotometricInterpretation == "MONOCHROME1":
        pixel_array = 1 - pixel_array

    # transform to hounsfield units
    pixel_array = pixel_array * dicom_image.RescaleSlope + dicom_image.RescaleIntercept

    # windowing
    window_center = int(dicom_image.WindowCenter)
    window_width = int(dicom_image.WindowWidth)
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    pixel_array = pixel_array.copy()
    pixel_array[pixel_array < img_min] = img_min
    pixel_array[pixel_array > img_max] = img_max

    # normalization
    pixel_array = np.zeros_like(pixel_array) if pixel_array.max() == pixel_array.min() else (pixel_array - pixel_array.min()) / (pixel_array.max() - pixel_array.min())
    return pixel_array

#  utils for image processing

In [31]:
train_labels = pd.read_csv(f"{Config.BASE_PATH}/train.csv")
#maby drop lower hu 

#input: entry to tree directory of train images
#output:  list of path to scan ( each scan  is a list with path to the images inside it )
def reshapePathsToScan(train_img_path):
    img_paths = []  
    for dirpath, dirnames, filenames in os.walk(train_img_path):
        if not filenames:
            continue  # skip directories that don't contain files
        scan = [os.path.join(dirpath, filename) for filename in filenames]
        img_paths.append(scan)
    return img_paths
def scaleSizeOfDataSet(img_paths,ratio):
    max_number_of_images=int(len(img_paths)*ratio)
    return img_paths[:max_number_of_images]
    
def process_image(path):
   # Open the image
    image = Image.open(path)
    
    # Convert to grayscale
    grayscale = image.convert('L')
    
    # Convert to numpy array and normalize
    grayscale_np = np.array(grayscale) / 255.0
    return grayscale_np



scansPaths=reshapePathsToScan(Config.TRAIN_IMG_PATH)
scansPaths=scaleSizeOfDataSet(scansPaths,1/30)

# CTScansDataSet

In [32]:
class CTScansDataSet(torch.utils.data.Dataset):
    def __init__(self, train_labels,scansPaths, current_fold,num_fold=5):
        self.train_labels=train_labels
        self.scansPaths  = scansPaths 
        #handle 5 fold validation 
        self.num_fold = num_fold
        self.current_fold = current_fold
        self.kfold = KFold(n_splits=num_fold)
        self.transform= torchvision.transforms.Compose([
                    torchvision.transforms.Resize((256, 256),antialias=True),
                    torchvision.transforms.RandomHorizontalFlip(),    # Random horizontal flip
                    torchvision.transforms.RandomRotation(45),
                    torchvision.transforms.RandomAffine(degrees=0, scale=(0.8, 1.2)),  # Apply zoom transformation
                ])
        
    def __len__(self):
        return len(self.scansPaths)

    def __getitem__(self, idx):
        #      dicom_images = select_elements_with_spacing(self.img_paths[idx],
#                                                     spacing = 2)
        #idx to scan-list of images
        images_paths=self.scansPaths[idx]
        patient_id = images_paths[0].split('/')[-3]
        images = [process_image(path) for path in images_paths]
        images=np.array(images)
        images=self.rescale_scan_to_group_3(images)
        # do augmentation to slices- move to 3d becouse we deal with resnet or other-pretraind model.
        images = self.transform(torch.from_numpy(images).float())
        images=standardize_depth(images)
        # get the labels from the df 
        labels = self.train_labels[self.train_labels.patient_id == int(patient_id)].values[0][1:-1]
        return images,labels,patient_id
        
    def rescale_scan_to_group_3(self,scan):
        padding = np.zeros((1,scan.shape[1], scan.shape[2]))
        if scan.shape[0]%3 ==1:
            scan=np.concatenate((padding,scan,padding),axis=0)
        elif  scan.shape[0]%3 ==2:  

            scan=np.concatenate((padding,scan),axis=0)
        return scan
    
    def split_fold(self):
        """
        Splits the dataset into training and validation subsets based on the current fold.
        
        Returns:
            tuple: A tuple containing the training and validation subsets.
        """
        #split across patient *-9
        fold_data = list(self.kfold.split(self.scansPaths))
        train_indices, val_indices = fold_data[self.current_fold]
        train_data = Subset(self, train_indices)
        val_data = Subset(self,val_indices)
        return train_data, val_data
    
    
#define k-fold partition for data -defualt 5 -data loaders

folds=[CTScansDataSet(train_labels,scansPaths, current_fold=i).split_fold() for i in range (Config.NUM_FOLDS)]
train_folds = [fold[0] for fold in folds]
val_folds = [fold[1] for fold in folds]
train_dataloaders= [DataLoader(train_folds[i],batch_size = Config.BATCH_SIZE, shuffle = True)  for i in range (Config.NUM_FOLDS) ]
val_dataloaders= [DataLoader(val_folds[i],batch_size = Config.BATCH_SIZE, shuffle = False)  for i in range (Config.NUM_FOLDS) ]

# MultiPathResNet

In [33]:
class MultiPathResNet(nn.Module):
    def __init__(self):
        super(MultiPathResNet, self).__init__()

#         self.backbone=torchvision.models.mobilenet_v3_large(weights='IMAGENET1K_V1')
        self.backbone=torchvision.models.resnet18(weights='IMAGENET1K_V1')
        for param in self.backbone.parameters():
            param.requires_grad = False
#         for param in self.backbone.layer4[-1].parameters():
#             param.requires_grad = True 
        self.backbone = nn.Sequential(*list( self.backbone.children())[:-2])
        self.ChannelReducer= nn.Conv2d(in_channels=512, out_channels=128, kernel_size=1)
        self.layer_normalization=nn.Sequential(torch.nn.BatchNorm3d(128),nn.ReLU())
# avgpool

# avgpool
# Instantiate the model

    def forward(self, x):
        batch_size = x.size(0)
        num_slices=x.size(1)//3
        outputs = []
        x=x.view(-1, 3, x.size(2), x.size(3))
        x=self.backbone(x)
        x=self.ChannelReducer(x).view(batch_size,128,num_slices,x.shape[-1],x.shape[-1])
        x=self.layer_normalization(x)
        return x
    
class Custom3DCNN(nn.Module):
    def __init__(self):
        super(Custom3DCNN, self).__init__()
        # First 3D convolution
        
        self.convBlock1=nn.Sequential( nn.Conv3d(128, 32, kernel_size=3,
                                                stride=(3, 1, 1), padding=(0, 1, 1)),nn.BatchNorm3d(32), nn.ReLU(),nn.Dropout(0.3))
        
        # Second 3D convolution
        self.convBlock2=nn.Sequential( nn.Conv3d(32, 8, kernel_size=3, stride=(3, 1, 1), padding=(0, 1, 1)),
                                       nn.BatchNorm3d(8), nn.ReLU(),nn.Dropout(0.3))
        
        # Fully connected layers
        self.global_avgpool = nn.AdaptiveAvgPool3d((8, None, None))
        self.flatten_size = 8 * 8 * 8 * 8  # Adjusted due to global average pooling
        self.Dense=nn.Sequential(nn.Linear(self.flatten_size, 512),nn.BatchNorm1d(512) ,nn.ReLU())
        
    def forward(self, x):
        batch_size=x.shape[0]
        x=self.convBlock1(x)
        x= self.convBlock2(x)
        x = self.global_avgpool(x)
        x = x.view(batch_size, self.flatten_size)  # Flatten
        x =self.Dense(x) 
        return x
           

# CT3DModel

In [34]:
class CT3DModel(nn.Module):
    def __init__(self,num_classes=13):
        super(CT3DModel, self).__init__()
        self.classifier = nn.Linear(512, num_classes)
        self.encoder=MultiPathResNet()
        self.decoder=Custom3DCNN()
    def forward(self, x):
            x=self.encoder(x)
            x=self.decoder(x)
            x = self.classifier(x)
            return(x)

# print(model)
# torchsummary.summary(model,(330 ,256, 256))

# MetricsCalculator

In [48]:
#class to calcilate metrics.

class MetricsCalculator:
    def __init__(self, mode = 'binary'):
        
        self.probabilities = []
        self.predictions = []
        self.targets = []
        
        self.mode = mode
        
    #logits batch_size*1*n, target batch_size*1*1 
    def update(self, logits, target):
        """
        Update the metrics calculator with predicted values and corresponding targets.
        
        Args:
            predicted (torch.Tensor): Predicted values.
            target (torch.Tensor): Ground truth targets.
        """
        probabilities = F.softmax(logits, dim = 1)
        predicted = torch.argmax(probabilities, dim=1)
        if self.mode == 'binary':
          #take positive class - for example if has extravision take all probabiltes that patient will have extravation and drop the probabiltes not.
          probabilities=probabilities[:,1]
            
        self.probabilities.extend(probabilities.detach().cpu().numpy())
        self.predictions.extend(predicted.detach().cpu().numpy())
        self.targets.extend(target.detach().cpu().numpy())
    
    def reset(self):
        """Reset the stored predictions and targets."""
        
        self.probabilities = []
        self.predictions = []
        self.targets = []
    
    def compute_accuracy(self):
        """
        Compute the accuracy metric.
        
        Returns:
            float: Accuracy.
        """
        return accuracy_score(self.targets, self.predictions)
    
    def compute_auc(self):
        """
        Compute the AUC (Area Under the Curve) metric.
        
        Returns:
            float: AUC.
        """
        if self.mode == 'multi':
            return roc_auc_score(self.targets, self.probabilities, multi_class = 'ovo', labels=[0, 1, 2])
    
        else:
            return roc_auc_score(self.targets, self.probabilities)


# initialize metrics objects
train_acc_bowel = MetricsCalculator('binary')
train_acc_extravasation = MetricsCalculator('binary')
train_acc_liver = MetricsCalculator('multi')
train_acc_kidney = MetricsCalculator('multi')
train_acc_spleen = MetricsCalculator('multi')

val_acc_bowel = MetricsCalculator('binary')
val_acc_extravasation = MetricsCalculator('binary')
val_acc_liver = MetricsCalculator('multi')
val_acc_kidney = MetricsCalculator('multi')
val_acc_spleen = MetricsCalculator('multi')


# main_functionmain_function

In [49]:
#main training function 
def main_function():

    model = CT3DModel(num_classes=13).to(device)
    #define loss
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5, verbose=True)
    loss_two_bowl = torch.nn.CrossEntropyLoss(weight = torch.tensor([1.0, 2.0])).to(device)
    loss_two_extra = torch.nn.CrossEntropyLoss(weight = torch.tensor([1.0, 6.0])).to(device)
    loss_three  = torch.nn.CrossEntropyLoss(label_smoothing = 0.05, weight = torch.tensor([1.0, 2.0, 4.0])).to(device)
    
    train_losses_ephocs = []
    val_losses_ephocs = []
    best_loss=np.inf
    
    for epoch in range(Config.EPOCHS):
        train_lose_ephoc =0
        val_lose_ephoc = 0
        train_loss=0
        val_loss=0
        print(f'Epoch: [{epoch+1}/{Config.EPOCHS}]')
        print(f'Fold: {epoch%5}')
        train_dataloader  = train_dataloaders[epoch%5]
        val_dataloader    = val_dataloaders[epoch%5]               
    
        for batch_idx, (images, labels,patient_id) in enumerate(train_dataloader):
            #################################### strat train ephoch  #########################################
            
            print(len(train_dataloader))
            print(batch_idx)
            optimizer.zero_grad()
            # Move data to GPU           
            images = images.to(device)  
            concatenated_categorial=convertOneHotToCategorial(labels).to(device)  
            bowel_batch_labels,extravasation_batch_labels=concatenated_categorial[:,0],concatenated_categorial[:,1]
            kidney_batch_labels,liver_batch_labels,spleen_batch_labels=concatenated_categorial[:,2],concatenated_categorial[:,3],concatenated_categorial[:,4]

            ## debug  ### 
            #----plot 10 fist scans
            # debug_plot_images(images,10,patient_id)
            # print(images.shape)
            # print(labels)
                 
            
            outputs = model(images)
            print(outputs)
            print(outputs[:, 0:2].dtype)  # Should be torch.float32 or similar      
            bowel_loss = loss_two_bowl(outputs[:, 0:2], bowel_batch_labels)
            extravasation_loss = loss_two_extra(outputs[:, 2:4], extravasation_batch_labels)
            kidney_loss = loss_three(outputs[:, 4:7], kidney_batch_labels)
            liver_loss = loss_three(outputs[:, 7:10], liver_batch_labels)
            spleen_loss = loss_three(outputs[:, 10:13], spleen_batch_labels)
            ###!!!!!!!!!To-do -add anyinjuery loss!!!!!!!!!!!!!!!
            train_loss = bowel_loss + extravasation_loss + kidney_loss + liver_loss + spleen_loss
            print(train_loss)
            # Backward pass
            
            train_loss.backward()
            # Update weights
            optimizer.step()
            #aggregate results for metrics
            train_lose_ephoc += train_loss.item()
            train_acc_bowel.update(outputs[:, 0:2], bowel_batch_labels)
            train_acc_extravasation.update(outputs[:, 2:4], extravasation_batch_labels)
            train_acc_kidney.update(outputs[:, 4:7], kidney_batch_labels)
            train_acc_liver.update(outputs[:, 7:10], liver_batch_labels)
            train_acc_spleen.update(outputs[:, 10:13], spleen_batch_labels)
            
        ############################################end train ephoch  ################################## 
        train_lose_ephoc=train_lose_ephoc/len(train_dataloader)
        train_losses_ephocs.append( train_lose_ephoc)
          
        model.eval()  # Set the model to evaluation mode    
        with torch.no_grad():
            for val_ind, (images, labels,patient_id) in enumerate(val_dataloader):  
                ##########################  strat eval  ephoch   #############################
                
                images = images.to(device)   
                #parse labels
                concatenated_categorial=convertOneHotToCategorial(labels).to(device)
                bowel_batch_labels,extravasation_batch_labels=concatenated_categorial[:,0],concatenated_categorial[:,1]
                kidney_batch_labels,liver_batch_labels,spleen_batch_labels=concatenated_categorial[:,2],concatenated_categorial[:,3],concatenated_categorial[:,4]
           
                outputs = model(images)
                bowel_loss = loss_two_bowl(outputs[:, 0:2], bowel_batch_labels)
                extravasation_loss = loss_two_extra(outputs[:, 2:4], extravasation_batch_labels)
                kidney_loss = loss_three(outputs[:, 4:7], kidney_batch_labels)
                liver_loss = loss_three(outputs[:, 7:10], liver_batch_labels)
                spleen_loss = loss_three(outputs[:, 10:13], spleen_batch_labels)
                
                ###!!!!!!!!!To-do -add any injuery loss!!!!!!!!!!!!!!!
                
                val_loss = bowel_loss + extravasation_loss + kidney_loss + liver_loss + spleen_loss 
                # calculate validation metrics    
                val_lose_ephoc += val_loss.item()
                val_acc_bowel.update(outputs[:, 0:2], bowel_batch_labels)
                val_acc_extravasation.update(outputs[:, 2:4], extravasation_batch_labels)
                val_acc_kidney.update(outputs[:, 4:7], kidney_batch_labels)
                val_acc_liver.update(outputs[:, 7:10], liver_batch_labels)
                val_acc_spleen.update(outputs[:, 10:13], spleen_batch_labels)
                
                ################################# end eval ephoch #############################
                
        # to get out of plauto         
        val_lose_ephoc=val_lose_ephoc/len(val_dataloader)
        scheduler.step(val_lose_ephoc)      
        val_losses_ephocs.append(val_lose_ephoc)     
        if val_loss <= best_loss:
          best_loss = val_loss
        #   torch.save(model.state_dict(), './drive/MyDrive/model_weights.pth')
        print(f"Epoch [{epoch+1}/{Config.EPOCHS}] - train Loss: {train_lose_ephoc:.4f} - Val Loss: {val_lose_ephoc:.4f}")
        # accuracy and auc data
        print(train_acc_bowel.targets)
        print(train_acc_bowel.predictions)
        print(train_acc_bowel.probabilities)
        metrics_data = [
                    ["Bowel", 
                        train_acc_bowel.compute_accuracy(),
                        val_acc_bowel.compute_accuracy(),
                        train_acc_bowel.compute_auc(),
                        val_acc_bowel.compute_auc()],
                    ["Extravasation", 
                        train_acc_extravasation.compute_accuracy(),
                        val_acc_extravasation.compute_accuracy(),
                        train_acc_extravasation.compute_auc(),
                        val_acc_extravasation.compute_auc()],
                    ["Liver", 
                        train_acc_liver.compute_accuracy(),
                        val_acc_liver.compute_accuracy(),
                        train_acc_liver.compute_auc(),
                        val_acc_liver.compute_auc()],
                    ["Kidney", 
                        train_acc_kidney.compute_accuracy(),
                        val_acc_kidney.compute_accuracy(),
                        train_acc_kidney.compute_auc(),
                        val_acc_kidney.compute_auc()],
                    ["Spleen", 
                        train_acc_spleen.compute_accuracy(),
                        val_acc_spleen.compute_accuracy(),
                        train_acc_spleen.compute_auc(),
                        val_acc_spleen.compute_auc()]
                ]
        # verbose
        print(tabulate(metrics_data, headers=["", "Train Acc", "Val Acc", "Train AUC", "Val AUC"]))
        #reset metrics
        train_acc_bowel.reset()
        train_acc_extravasation.reset()
        train_acc_liver.reset()
        train_acc_kidney.reset()
        train_acc_spleen.reset()
        val_acc_bowel.reset()
        val_acc_extravasation.reset()
        val_acc_liver.reset()
        val_acc_kidney.reset()
        val_acc_spleen.reset()     
        
def convertOneHotToCategorial(labels):
        bowel_batch_labels = np.argmax(labels[:,0:2],axis=1 ,keepdims = True).squeeze(-1)
        extravasation_batch_labels = np.argmax(labels[:,2:4],axis=1 ,keepdims = True).squeeze(-1)
        kidney_batch_labels = np.argmax(labels[:,4:7],axis=1 ,keepdims = True).squeeze(-1)
        liver_batch_labels = np.argmax(labels[:,7:10],axis=1, keepdims = True).squeeze(-1)
        spleen_batch_labels = np.argmax(labels[:,10:],axis=1, keepdims = True).squeeze(-1)
        concatenated_categorial =np.vstack((bowel_batch_labels, extravasation_batch_labels, kidney_batch_labels,liver_batch_labels,spleen_batch_labels)).T
        concatenated_categorial = torch.tensor(concatenated_categorial)
        return concatenated_categorial

def debug_plot_images(images,num_scans,patient_id):
    
    first_scans = images[:num_scans]
            # Plot the firsts scans
    for scan_idx, scan in enumerate(first_scans):
                print(f'patient id {patient_id[scan_idx]}')
                num_images = scan.shape[0]
                plt.figure(figsize=(20, 20))  # Adjust the figure size as needed
                plt.suptitle(f"Scan {scan_idx + 1}")
                for image_idx in range(num_images):
                    plt.subplot(30, 10, image_idx + 1)
                    plt.imshow(scan[image_idx], cmap='gray')  # Assuming grayscale images
                    plt.axis('off')  # Turn off axis labels

                plt.show()
    
main_function()


Epoch: [1/3]
Fold: 0
5
0
tensor([[-3.7712e-02, -6.8837e-01,  9.1414e-02, -2.5361e-01, -3.7734e-01,
          2.7402e-01, -1.6277e-01,  2.8522e-01, -7.9483e-01, -1.4469e-01,
          3.6017e-01,  5.5910e-01, -1.5474e-01],
        [-7.0519e-01,  6.2444e-02, -2.4509e-01, -8.3833e-01,  1.2059e+00,
          1.2006e-01, -4.5751e-02,  2.9378e-01, -7.9237e-01,  2.6466e-01,
         -9.8815e-02,  6.9820e-01, -1.8758e-01],
        [ 9.7030e-03, -1.0763e-01, -3.6144e-01, -2.4421e-01,  5.2821e-02,
          4.2737e-01, -2.1990e-01,  5.1859e-02, -2.3752e-01,  6.6336e-01,
          8.5738e-01,  1.0944e+00,  2.0961e-01],
        [-5.1262e-01,  9.8902e-02, -5.2313e-01, -1.8125e-01,  2.1161e-01,
          1.0120e-01, -2.0119e-01,  8.9830e-02,  5.2725e-02, -1.0685e-01,
          1.2193e-01,  1.7421e-01,  2.8207e-01],
        [ 2.6509e-01, -1.5821e-01, -2.9792e-01, -4.2572e-01,  5.0773e-01,
          9.2558e-01, -1.7255e-03,  3.0481e-01, -1.9319e-01, -2.2248e-01,
          6.8934e-01,  4.3972e-01,  3.0

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
torch.cuda.empty_cache()

In [ ]:
torch.save(model.state_dict(), './drive/MyDrive/model_ResNet18+3DConv_weights.pth')

In [ ]:
render_window = vtk.vtkRenderWindow()
render_window_interactor = vtk.vtkRenderWindowInteractor()
render_window_interactor.SetRenderWindow(render_window)

In [ ]:
# Plot training and validation progress
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses_ephocs, label='Train')
plt.plot(val_losses_ephocs, label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')

plt.tight_layout()
plt.show()

In [ ]:
# model = torch.load('model.pth')
# model.load_state_dict(torch.load('./drive/MyDrive/DATA/model_weights.pth'))

# trt_model = torch_tensorrt.compile(model, 
#     inputs= [torch_tensorrt.Input((1, 3, 224, 224))],
#     enabled_precisions= { torch_tensorrt.dtype.half} # Run with FP16
# )